In [4]:
import pandas as pd, re

In [5]:
sarcasm_df = pd.read_csv('sarcasm.csv')

sarcasm_df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [17]:
pattern = r"^https://([^/]+)/"

sarcasm_df['article_host'] = sarcasm_df['article_host'].apply(lambda row: re.search(pattern, row)[1])

print(sarcasm_df.head())

KeyError: 'article_host'